<table align="center">
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/andrew-nash/CS6421-labs/blob/main/Lab7.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/andrew-nash/CS6421-labs/blob/main/Lab7.ipynb">
        <img src="https://i.ibb.co/xfJbPmL/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Anomaly Detection Autoencoders

Source: https://www.tensorflow.org/tutorials/generative/autoencoder#third_example_anomaly_detection

In this lab we will look at modelling non-image data with autoencoders - specifically, we will be taking ECG (Electrocardiogram) signals, and uing an autoencoder to identify potential abnormalities in the signals

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split


# The Data

We will be using the [ECG5000 Dataset](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000). This countains 5000 ECG signals each of 140 samples. Each signal has a label indicating whether it contains normal or abnormal behaviour.

In [ ]:
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

In [48]:
# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, valid_data, train_labels, valid_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

In [ ]:
labels

The only pre-processing needed will be to scalem the data to be between 0 and 1

In [50]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
valid_data = (valid_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
valid_data = tf.cast(valid_data, tf.float32)

In [51]:
# convert the 1s and 0s to True/False
train_labels = train_labels.astype(bool)
valid_labels = valid_labels.astype(bool)

# use a handy trick of numpy arrays and tf tensors
#    train_data[ [True,False,True,False,False,...] ] will return the values of train_data
#                                                    at indices that correspond to True
normal_train_data = train_data[train_labels]
normal_valid_data = valid_data[valid_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_valid_data = valid_data[~valid_labels]

In [ ]:
plt.grid()
plt.plot(np.arange(140), normal_train_data[0])
plt.title("A Normal ECG")
plt.show()

In [ ]:
plt.grid()
plt.plot(np.arange(140), anomalous_train_data[0])
plt.title("An Anomalous ECG")
plt.show()

# Modelling

We are going to approach this problem in what might perhaps be a surprising manner.

We will **not** fit a regression/classification deep model onto the data directly.

Instead, we will fit an autoecnoder to the signals and attempt to reconstruct its inputs.

This can be considered to be a model that tries as best as possible to learn a *"normal"* model of ECG signal behaviour. Then, we will compare the *actual* signals, to their reconstruction - our assumption is that deviations from the reconstructions are caused by anomalous behavior.

<strong>What is the benefit of this over a more traditional approach?</strong>

## Creating the reconstruction autoencoder model

In [ ]:
!pip install wandb -qU
%load_ext tensorboard
%tensorboard --logdir tboard

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
wandb.login()

In [ ]:

run_name="Dense Autoencoder"
tensorboard_callback = tf.keras.callbacks.TensorBoard(f"./tboard/{run_name}", histogram_freq=1)
wandb.init(
        project = "Lab7",
        name =   run_name)

class AnomalyDetector(tf.keras.models.Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(16, activation="relu"),
      tf.keras.layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      tf.keras.layers.Dense(16, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.fit(normal_train_data, normal_train_data,
          epochs=20,
          batch_size=512,
          validation_data=(valid_data, valid_data),
          shuffle=True, callbacks=[tensorboard_callback,WandbMetricsLogger()])

Let us now look at the reconstruction of a normal (non-anaomalous) ECG signal

In [ ]:
encoded_data = autoencoder.encoder(normal_valid_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.plot(normal_valid_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], normal_valid_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

... compared to an anomalous signal:

In [ ]:
encoded_data = autoencoder.encoder(anomalous_valid_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.plot(anomalous_valid_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], anomalous_valid_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

## Detecting the anomalies

Detect anomalies by calculating whether the reconstruction loss is greater than a fixed threshold. In this tutorial, you will calculate the mean average error for normal examples from the training set, then classify future examples as anomalous if the reconstruction error is higher than one standard deviation from the training set.

Plot the reconstruction error on normal ECGs from the training set

In [ ]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

There are many heuristics for choosing this threshold.

Next, look at the distribution of reconstruction errors for the validation data

In [ ]:
reconstructions = autoencoder.predict(anomalous_valid_data)
valid_loss = tf.keras.losses.mae(reconstructions, anomalous_valid_data)

plt.hist(valid_loss[None, :], bins=50)
plt.xlabel("Validation loss")
plt.ylabel("No of examples")
plt.show()

In [18]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
preds = predict(autoencoder, valid_data, threshold)
print_stats(preds, valid_labels)

## CNN Autoencoders

Now, instead of a Dense model, lets introduce a CNN model and see if we can improve performance.

We are working with a time-series vector rather than a 2D image (or rank-3 tensor for a 2D image with multiple colour channels). This means that all of our convolutional kernels will have height of 1. In this case, we could use Conv2D with 1 values for all height hyper-parameters. For simplicity, TensorFlow exposes a Con1D layer that does this for us automatically to make out code cleaner.

However, this is onl a wrapper on Conv2D - our data still needs to be converted to a 2D matix (of height 1)

In [108]:
anomalous_train_data_2d = tf.expand_dims(anomalous_train_data, -1)
normal_train_data_2d = tf.expand_dims(normal_train_data, -1)
valid_data_2d = tf.expand_dims(valid_data, -1)
anomalous_valid_data_2d = tf.expand_dims(anomalous_valid_data, -1)
normal_valid_data_2d = tf.expand_dims(normal_valid_data, -1)

In [ ]:

run_name="CNN Autoencoder"
tensorboard_callback = tf.keras.callbacks.TensorBoard(f"./tboard/{run_name}", histogram_freq=1)
#wandb.init(
#        project = "Lab7",
#        name =   run_name)

class AnomalyDetector(tf.keras.models.Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
        # initial dimension is 140
      tf.keras.layers.Conv1D(filters=40,kernel_size=64,padding='same',activation="relu", input_shape=(140,1)),
      tf.keras.layers.Conv1D(filters=16,kernel_size=32,padding='same',activation="relu"),
      tf.keras.layers.MaxPooling1D(2, padding="valid"),
        # pooled to length 70
      tf.keras.layers.Conv1D(filters=16,kernel_size=8,padding='same',activation="relu"),
      tf.keras.layers.MaxPooling1D(7, padding="same"),
        # pooled to length 10
    ])

    self.decoder = tf.keras.Sequential([
      tf.keras.layers.Conv1D(filters=16,kernel_size=8,padding='same',activation="relu"),
      tf.keras.layers.UpSampling1D(7),
      # upscaled to 70
      tf.keras.layers.Conv1D(filters=16,kernel_size=32,activation="relu",padding="same"),
      tf.keras.layers.UpSampling1D(2),
      # upscalaed to 140
      tf.keras.layers.Conv1D(filters=16,kernel_size=32,padding='same',activation="relu"),
      tf.keras.layers.Conv1D(filters=1,kernel_size=64,padding='same',activation="relu"),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()
autoencoder.compile(optimizer='adam', loss='mae')

autoencoder.fit(normal_train_data_2d, normal_train_data_2d,
          epochs=20,
          batch_size=512,
          validation_data=(valid_data_2d, valid_data_2d),
          shuffle=True, callbacks=[tensorboard_callback,WandbMetricsLogger()])

### Sample Predictions

In [ ]:
encoded_data = autoencoder.encoder(normal_train_data_2d).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy().reshape(-1,140)

plt.plot(normal_valid_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], normal_valid_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
encoded_data = autoencoder.encoder(anomalous_valid_data_2d).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy().reshape(-1,140)

plt.plot(anomalous_valid_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], anomalous_valid_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

### Distributions of errors

In [ ]:
reconstructions = tf.squeeze(autoencoder.predict(normal_train_data_2d),2)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [140]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  0.0334326


In [ ]:
reconstructions = tf.squeeze(autoencoder.predict(anomalous_valid_data_2d),2)
valid_loss = tf.keras.losses.mae(reconstructions, anomalous_valid_data)

plt.hist(valid_loss[None, :], bins=50)
plt.xlabel("Validation loss")
plt.ylabel("No of examples")
plt.show()

In [142]:
def predict(model, data, threshold):
  reconstructions = tf.squeeze(model(data),2)
  loss = tf.keras.losses.mae(reconstructions, tf.squeeze(data,2))
  return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
preds = predict(autoencoder, valid_data_2d, threshold)
print_stats(preds, valid_labels)